In [1]:
# Import Modules 
import pandas as pd
import numpy as np
import math
import datetime
import statistics

import os
from tqdm import tqdm
from tqdm import trange

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FamaMacBeth
from linearmodels.panel import PooledOLS
import statsmodels.api as sm  # Adding constant
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
# from jupyterthemes import jtplot
# jtplot.style()

import warnings
warnings.filterwarnings('ignore')

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics(ip))

# from notify_run import Notify
# notify = Notify()
# notify.register()

<IPython.core.display.Javascript object>

## Define Function for Collecting Panel Analysis Results

In [2]:
# import modules
from statsmodels.compat.python import (lrange, iterkeys, iteritems, lzip,
                                       reduce, itervalues, zip, string_types,
                                       range)
from collections import OrderedDict
import statsmodels

import datetime
import textwrap
from statsmodels.iolib.table import SimpleTable
from statsmodels.iolib.tableformatting import fmt_latex, fmt_txt

In [3]:
class Summary_panel(object):
    def __init__(self):
        self.tables = []
        self.settings = []
        self.extra_txt = []
        self.title = None

    def __str__(self):
        return self.as_text()

    def __repr__(self):
        return str(type(self)) + '\n"""\n' + self.__str__() + '\n"""'

    def _repr_html_(self):
        '''Display as HTML in IPython notebook.'''
        return self.as_html()
    
    def as_text(self):
        '''Generate ASCII Summary Table
        '''

        tables = self.tables
        settings = self.settings
        title = self.title
        extra_txt = self.extra_txt

        pad_col, pad_index, widest = _measure_tables(tables, settings)

        rule_equal = widest * '='
        #TODO: this isn't used anywhere?
        rule_dash = widest * '-'

        simple_tables = _simple_tables(tables, settings, pad_col, pad_index)
        tab = [x.as_text() for x in simple_tables]

        tab = '\n'.join(tab)
        tab = tab.split('\n')
        tab[0] = rule_equal
        tab.append(rule_equal)
        tab = '\n'.join(tab)

        if title is not None:
            title = title
            if len(title) < widest:
                title = ' ' * int(widest/2 - len(title)/2) + title
        else:
            title = ''

        txt = [textwrap.wrap(x, widest) for x in extra_txt]
        txt = ['\n'.join(x) for x in txt]
        txt = '\n'.join(txt)

        out = '\n'.join([title, tab, txt])

        return out

    def as_html(self):
        '''Generate HTML Summary Table
        '''

        tables = self.tables
        settings = self.settings
        #TODO: this isn't used anywhere
        title = self.title

        simple_tables = _simple_tables(tables, settings)
        tab = [x.as_html() for x in simple_tables]
        tab = '\n'.join(tab)

        return tab

    def as_latex(self):
        '''Generate LaTeX Summary Table
        '''
        tables = self.tables
        settings = self.settings
        title = self.title
        if title is not None:
            title = '\\caption{' + title + '} \\\\'
        else:
            title = '\\caption{}'

        simple_tables = _simple_tables(tables, settings)
        tab = [x.as_latex_tabular() for x in simple_tables]
        tab = '\n\\hline\n'.join(tab)

        out = '\\begin{table}', title, tab, '\\end{table}'
        out = '\n'.join(out)
        return out
    
    def add_base(self, results, yname, xname, 
                 alpha=0.05, float_format="%.4f", title=None):
        '''Try to construct a basic summary instance.

        Parameters
        ----------
        results : Model results instance
        alpha : float
            significance level for the confidence intervals (optional)
        float_formatting: string
            Float formatting for summary of parameters (optional)
        title : string
            Title of the summary table (optional)
        xname : List of strings of length equal to the number of parameters
            Names of the independent variables (optional)
        yname : string
            Name of the dependent variable (optional)
        '''

        param = summary_params(results, alpha=alpha, use_t=True)
        info = summary_model(results)
        if xname is not None:
            param.index = xname
        if yname is not None:
            info['Dependent Variable:'] = yname
        self.add_dict(info, align='l')
        self.add_df(param, float_format=float_format)
        self.add_title(title=title, results=results)

    def add_df(self, df, index=True, header=True, float_format='%.4f',
               align='r'):
        '''Add the contents of a DataFrame to summary table

        Parameters
        ----------
        df : DataFrame
        header: bool
            Reproduce the DataFrame column labels in summary table
        index: bool
            Reproduce the DataFrame row labels in summary table
        float_format: string
            Formatting to float data columns
        align : string
            Data alignment (l/c/r)
        '''

        settings = {'index': index, 'header': header,
                    'float_format': float_format, 'align': align}
        self.tables.append(df)
        self.settings.append(settings)

    def add_array(self, array, align='r', float_format="%.4f"):
        '''Add the contents of a Numpy array to summary table

        Parameters
        ----------
        array : numpy array (2D)
        float_format: string
            Formatting to array if type is float
        align : string
            Data alignment (l/c/r)
        '''

        table = pd.DataFrame(array)
        self.add_df(table, index=False, header=False,
                    float_format=float_format, align=align)

    def add_dict(self, d, ncols=2, align='l', float_format="%.4f"):
        '''Add the contents of a Dict to summary table

        Parameters
        ----------
        d : dict
            Keys and values are automatically coerced to strings with str().
            Users are encouraged to format them before using add_dict.
        ncols: int
            Number of columns of the output table
        align : string
            Data alignment (l/c/r)
        '''

        keys = [_formatter(x, float_format) for x in iterkeys(d)]
        vals = [_formatter(x, float_format) for x in itervalues(d)]
        data = np.array(lzip(keys, vals))

        if data.shape[0] % ncols != 0:
            pad = ncols - (data.shape[0] % ncols)
            data = np.vstack([data, np.array(pad * [['', '']])])

        data = np.split(data, ncols)
        data = reduce(lambda x, y: np.hstack([x, y]), data)
        self.add_array(data, align=align)

    def add_text(self, string):
        '''Append a note to the bottom of the summary table. In ASCII tables,
        the note will be wrapped to table width. Notes are not indendented.
        '''
        self.extra_txt.append(string)

    def add_title(self, title=None, results=None):
        '''Insert a title on top of the summary table. If a string is provided
        in the title argument, that string is printed. If no title string is
        provided but a results instance is provided, statsmodels attempts
        to construct a useful title automatically.
        '''
        if isinstance(title, string_types):
            self.title = title
        else:
            try:
                model = results.model.__class__.__name__
#                 if model in _model_types:
#                     model = _model_types[model]
                self.title = 'Results: ' + model
            except:
                self.title = ''

In [4]:
def summary_model_panel(results):
    '''Create a dict with information about the model'''
    
    def time_now(*args, **kwds):
        now = datetime.datetime.now()
        return now.strftime('%Y-%m-%d %H:%M')
    info = OrderedDict()
    info['Model:'] = lambda x: x.model.__class__.__name__
#     info['Model Family:'] = lambda x: x.family.__class.__name__
#     info['Link Function:'] = lambda x: x.family.link.__class__.__name__
    info['Dependent Variable:'] = lambda x: ' '.join(map(str, results.model.dependent.vars))
    info['Date:'] = time_now
    info['No. Observations:'] = lambda x: "%#6d" % x.nobs
    info['Df Model:'] = lambda x: "%#6d" % x.df_model
    info['Df Residuals:'] = lambda x: "%#6d" % x.df_resid
#     info['Converged:'] = lambda x: x.mle_retvals['converged']
#     info['No. Iterations:'] = lambda x: x.mle_retvals['iterations']
    info['Method:'] = lambda x: x.method
#     info['Norm:'] = lambda x: x.fit_options['norm']
#     info['Scale Est.:'] = lambda x: x.fit_options['scale_est']
#     info['Cov. Type:'] = lambda x: x.fit_options['cov']
    info['R-squared:'] = lambda x: "%#8.3f" % x.rsquared
#     info['Adj. R-squared:'] = lambda x: "%#8.3f" % x.rsquared_adj
#     info['Pseudo R-squared:'] = lambda x: "%#8.3f" % x.prsquared
#     info['AIC:'] = lambda x: "%8.4f" % x.aic
#     info['BIC:'] = lambda x: "%8.4f" % x.bic
    info['Log-Likelihood:'] = lambda x: "%#8.5g" % x.loglik
#     info['LL-Null:'] = lambda x: "%#8.5g" % x.llnull
#     info['LLR p-value:'] = lambda x: "%#8.5g" % x.llr_pvalue
#     info['Deviance:'] = lambda x: "%#8.5g" % x.deviance
#     info['Pearson chi2:'] = lambda x: "%#6.3g" % x.pearson_chi2
    info['F-statistic:'] = lambda x: "%#8.4g" % x.f_statistic
#     info['Prob (F-statistic):'] = lambda x: "%#6.3g" % x.f_pvalue
#     info['Scale:'] = lambda x: "%#8.5g" % x.scale
    out = OrderedDict()
    for key, func in iteritems(info):
        try:
            out[key] = func(results)
        # NOTE: some models don't have loglike defined (RLM), so that's NIE
        except (AttributeError, KeyError, NotImplementedError):
            pass
    return out

In [5]:
def summary_params_panel(results, yname=None, xname=None, alpha=.05, use_t=True,
                   skip_header=False, float_format="%.4f"):
    '''create a summary table of parameters from results instance

    Parameters
    ----------
    res : results instance
        some required information is directly taken from the result
        instance
    yname : string or None
        optional name for the endogenous variable, default is "y"
    xname : list of strings or None
        optional names for the exogenous variables, default is "var_xx"
    alpha : float
        significance level for the confidence intervals
    use_t : bool
        indicator whether the p-values are based on the Student-t
        distribution (if True) or on the normal distribution (if False)
    skip_headers : bool
        If false (default), then the header row is added. If true, then no
        header row is added.
    float_format : string
        float formatting options (e.g. ".3g")

    Returns
    -------
    params_table : SimpleTable instance
    '''

    if isinstance(results, tuple):
        results, params, std_errors, tstats, pvalues, conf_int = results
    else:
        params = results.params
        std_errors = results.std_errors
        tstats = results.tstats
        pvalues = results.pvalues
        conf_int = results.conf_int(alpha)

    data = np.array([params, std_errors, tstats, pvalues]).T
    data = np.hstack([data, conf_int])
    data = pd.DataFrame(data)

    if use_t:
        data.columns = ['Coef.', 'Std.Err.', 't', 'P>|t|',
                        '[' + str(alpha/2), str(1-alpha/2) + ']']
    else:
        data.columns = ['Coef.', 'Std.Err.', 'z', 'P>|z|',
                        '[' + str(alpha/2), str(1-alpha/2) + ']']

    if not xname:
        data.index = results.model.exog_names
    else:
        data.index = xname

    return data

In [6]:
def _col_params_panel(result, yname=None, xname=None, float_format='%.4f', stars=True):
    '''Stack coefficients and standard errors in single column
    '''
    # Extract parameters
    res = summary_params_panel(result, yname=yname, xname=xname)
    # Format float
    for col in res.columns[:2]:
        res[col] = res[col].apply(lambda x: float_format % x)
    # Std.Errors in parentheses
    res.ix[:, 1] = '(' + res.ix[:, 1] + ')'
    # Significance stars
    if stars:
        idx = res.ix[:, 3] < .1
        res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
        idx = res.ix[:, 3] < .05
        res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
        idx = res.ix[:, 3] < .01
        res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
    # Stack Coefs and Std.Errors
    res = res.ix[:, :2]
    res = res.stack()
    res = pd.DataFrame(res)
    res.columns = result.model.dependent.vars
    return res

In [7]:
def _col_info_panel(result, info_dict=None):
    '''Stack model info in a column
    '''

    if info_dict is None:
        info_dict = {}
    out = []
    index = []
    for i in info_dict:
        if isinstance(info_dict[i], dict):
            # this is a specific model info_dict, but not for this result...
            continue
        try:
            out.append(info_dict[i](result))
        except:
            out.append('')
        index.append(i)
    out = pd.DataFrame({' '.join(map(str, result.model.dependent.vars)): out}, index=index)
    return out

In [8]:
def _make_unique(list_of_names):
    if len(set(list_of_names)) == len(list_of_names):
        return list_of_names
    # pandas does not like it if multiple columns have the same names
    from collections import defaultdict
    name_counter = defaultdict(str)
    header = []
    for _name in list_of_names:
        name_counter[_name] += "I"
        header.append(_name+" " + name_counter[_name])
    return header

In [9]:
def summary_col_panel(results, yname=None, xname=None, float_format='%.4f', 
                model_names=[], stars=False, info_dict=None, regressor_order=[]):
    """
    Summarize multiple results instances side-by-side (coefs and SEs)

    Parameters
    ----------
    results : statsmodels results instance or list of result instances
    float_format : string
        float format for coefficients and standard errors
        Default : '%.4f'
    model_names : list of strings of length len(results) if the names are not
        unique, a roman number will be appended to all model names
    stars : bool
        print significance stars
    info_dict : dict
        dict of lambda functions to be applied to results instances to retrieve
        model info. To use specific information for different models, add a
        (nested) info_dict with model name as the key.
        Example: `info_dict = {"N":..., "R2": ..., "OLS":{"R2":...}}` would
        only show `R2` for OLS regression models, but additionally `N` for
        all other results.
        Default : None (use the info_dict specified in
        result.default_model_infos, if this property exists)
    regressor_order : list of strings
        list of names of the regressors in the desired order. All regressors
        not specified will be appended to the end of the list.
    """

    if not isinstance(results, list):
        results = [results]

    cols = [_col_params_panel(x, yname=yname, xname=xname, 
                              float_format=float_format,  stars=stars) for x in
            results]

    # Unique column names (pandas has problems merging otherwise)
    if model_names:
        colnames = _make_unique(model_names)
    else:
        colnames = _make_unique([x.columns[0] for x in cols])
    for i in range(len(cols)):
        cols[i].columns = [colnames[i]]

    merg = lambda x, y: x.merge(y, how='outer', right_index=True,
                                left_index=True)
    summ = reduce(merg, cols)

    if regressor_order:
        varnames = summ.index.get_level_values(0).tolist()
        ordered = [x for x in regressor_order if x in varnames]
        unordered = [x for x in varnames if x not in regressor_order + ['']]
        order = ordered + list(np.unique(unordered))

        f = lambda idx: sum([[x + 'coef', x + 'stde'] for x in idx], [])
        summ.index = f(np.unique(varnames))
        summ = summ.reindex(f(order))
        summ.index = [x[:-4] for x in summ.index]

    idx = pd.Series(lrange(summ.shape[0])) % 2 == 1
    summ.index = np.where(idx, '', summ.index.get_level_values(0))

    # add infos about the models.
    if info_dict:
        cols = [_col_info_panel(x, info_dict.get(x.model.__class__.__name__,
                                           info_dict)) for x in results]
    else:
        cols = [_col_info_panel(x, getattr(x, "default_model_infos", None)) for x in
                results]
    # use unique column names, otherwise the merge will not succeed
    for df , name in zip(cols, _make_unique([df.columns[0] for df in cols])):
        df.columns = [name]
    merg = lambda x, y: x.merge(y, how='outer', right_index=True,
                                left_index=True)
    info = reduce(merg, cols)
    dat = pd.DataFrame(np.vstack([summ, info]))  # pd.concat better, but error
    dat.columns = summ.columns
    dat.index = pd.Index(summ.index.tolist() + info.index.tolist())
    summ = dat

    summ = summ.fillna('')

    smry = Summary_panel()
    smry.add_df(summ, header=True, align='l')
    smry.add_text('Standard errors in parentheses.')
    if stars:
        smry.add_text('* p<.1, ** p<.05, ***p<.01')

    return smry

In [10]:
def _measure_tables(tables, settings):
    '''Compare width of ascii tables in a list and calculate padding values.
    We add space to each col_sep to get us as close as possible to the
    width of the largest table. Then, we add a few spaces to the first
    column to pad the rest.
    '''

    simple_tables = _simple_tables(tables, settings)
    tab = [x.as_text() for x in simple_tables]

    length = [len(x.splitlines()[0]) for x in tab]
    len_max = max(length)
    pad_sep = []
    pad_index = []

    for i in range(len(tab)):
        nsep = tables[i].shape[1] - 1
        pad = int((len_max - length[i]) / nsep)
        pad_sep.append(pad)
        len_new = length[i] + nsep * pad
        pad_index.append(len_max - len_new)

    return pad_sep, pad_index, max(length)

def _formatter(element, float_format='%.4f'):
    try:
        out = float_format % element
    except:
        out = str(element)
    return out.strip()

def _df_to_simpletable(df, align='r', float_format="%.4f", header=True,
                       index=True, table_dec_above='-', table_dec_below=None,
                       header_dec_below='-', pad_col=0, pad_index=0):
    dat = df.copy()
    dat = dat.applymap(lambda x: _formatter(x, float_format))
    if header:
        headers = [str(x) for x in dat.columns.tolist()]
    else:
        headers = None
    if index:
        stubs = [str(x) + int(pad_index) * ' ' for x in dat.index.tolist()]
    else:
        dat.ix[:, 0] = [str(x) + int(pad_index) * ' ' for x in dat.ix[:, 0]]
        stubs = None
    st = SimpleTable(np.array(dat), headers=headers, stubs=stubs,
                     ltx_fmt=fmt_latex, txt_fmt=fmt_txt)
    st.output_formats['latex']['data_aligns'] = align
    st.output_formats['txt']['data_aligns'] = align
    st.output_formats['txt']['table_dec_above'] = table_dec_above
    st.output_formats['txt']['table_dec_below'] = table_dec_below
    st.output_formats['txt']['header_dec_below'] = header_dec_below
    st.output_formats['txt']['colsep'] = ' ' * int(pad_col + 1)
    return st

def _simple_tables(tables, settings, pad_col=None, pad_index=None):
    simple_tables = []
    float_format = '%.4f'
    if pad_col is None:
        pad_col = [0] * len(tables)
    if pad_index is None:
        pad_index = [0] * len(tables)
    for i, v in enumerate(tables):
        index = settings[i]['index']
        header = settings[i]['header']
        align = settings[i]['align']
        simple_tables.append(_df_to_simpletable(v, align=align,
                                                float_format=float_format,
                                                header=header, index=index,
                                                pad_col=pad_col[i],
                                                pad_index=pad_index[i]))
    return simple_tables

## Collecting Panel Analysis Results for All Intervals

In [36]:
interval = '10'

In [41]:
phase_1 = pd.read_csv('/Volumes/LaCie/2020July/panel_analysis/%s/Phase_1.csv'%interval)
phase_1.iloc[:,0] = pd.to_datetime(pd.Series(phase_1.iloc[:,0]))
mi_p1 = phase_1.set_index(['Code','DateTime'])
mi_p1['Post1*G1'] = mi_p1['Post 1']*mi_p1['G1']
mi_p1['Post1*G2'] = mi_p1['Post 1']*mi_p1['G2']
mi_p1['Post1*G3'] = mi_p1['Post 1']*mi_p1['G3']
mi_p1['Post1*G4'] = mi_p1['Post 1']*mi_p1['G4']
mi_p1['Post1*G5'] = mi_p1['Post 1']*mi_p1['G5']
mi_p1['Post1*G6'] = mi_p1['Post 1']*mi_p1['G6']

In [52]:
exog_var_1 = ['Post1*G1', 'Post1*G2', 'Post1*G3', 
              'Post1*G4', 'Post1*G5', 
              'Lag Return-%s'%interval]#, 'Post1*G6','Volume Yen-%s'%interval]
ind_var=sm.add_constant(mi_p1[exog_var_1])
# ind_var = mi_p1[exog_var_1]

In [53]:
RE_mod_1 = RandomEffects(mi_p1['Illiquidity-%s'%interval], ind_var)
print(RE_mod_1.fit())

                        RandomEffects Estimation Summary                        
Dep. Variable:         Illiquidity-10   R-squared:                        0.0006
Estimator:              RandomEffects   R-squared (Between):              0.0527
No. Observations:             6304360   R-squared (Within):               0.0006
Date:                Tue, Jul 07 2020   R-squared (Overall):              0.0040
Time:                        18:20:33   Log-likelihood                  7.82e+06
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      670.84
Entities:                          87   P-value                           0.0000
Avg Obs:                    7.246e+04   Distribution:               F(6,6304353)
Min Obs:                     6.97e+04                                           
Max Obs:                    7.252e+04   F-statistic (robust):             670.84
                            

In [47]:
a = ind_var.dropna()

In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(a.values, i)\
                     for i in range(a.shape[1])]
vif["features"] = a.columns

In [49]:
vif.round(1)

,VIF Factor,features
0,2.3,const
1,1.1,Post1*G1
2,1.1,Post1*G2
3,1.1,Post1*G3
4,1.1,Post1*G4
5,1.0,Post1*G5
6,1.0,Post1*G6
7,1.0,Lag Return-10
8,1.0,Volume Yen-10


In [54]:
exog_var_2 = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'Lag Return-%s'%interval]
ind_var_2 = sm.add_constant(mi_p1[exog_var_2])
b = ind_var_2.dropna()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(b.values, i)\
                     for i in range(b.shape[1])]
vif["features"] = b.columns
vif.round(1)

,VIF Factor,features
0,0.0,const
1,inf,G1
2,inf,G2
3,inf,G3
4,inf,G4
5,inf,G5
6,inf,G6
7,1.0,Lag Return-10


In [56]:
exog_var_3 = ['G1', 'G2', 'G3', 'G4', 'G5', 'Lag Return-%s'%interval]
ind_var_3 = sm.add_constant(mi_p1[exog_var_3])
c = ind_var_3.dropna()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(c.values, i)\
                     for i in range(c.shape[1])]
vif["features"] = c.columns
vif.round(1)

,VIF Factor,features
0,87.0,const
1,17.7,G1
2,21.5,G2
3,11.9,G3
4,9.7,G4
5,5.7,G5
6,1.0,Lag Return-10


In [13]:
# Post 1 
result_post1 = []
pbar = tqdm(['05'])#,'10','15','30','45','60'])
for t in pbar:
    phase_1 = pd.read_csv('/Volumes/LaCie/2020July/panel_analysis/%s/Phase_1.csv'%t)
    phase_1.iloc[:,0] = pd.to_datetime(pd.Series(phase_1.iloc[:,0]))
    mi_p1 = phase_1.set_index(['Code','DateTime'])
    exog_var_1 = ['Post 1']
    ind_var = sm.add_constant(mi_p1[exog_var_1])    # With constant
#     ind_var=mi_p1[exog_var_1]    # Without constant
    RE_mod_1 = RandomEffects(mi_p1['Illiquidity-%s'%t], ind_var)
    RE_mod_1_res = RE_mod_1.fit()
    result_post1.append(RE_mod_1_res)
    
exog_var = ['Intercept','Post 1']
dfoutput_post1 = summary_col_panel(result_post1, xname=exog_var,stars=True)
dfoutput_post1

100%|██████████| 1/1 [00:28<00:00, 28.73s/it]


ZeroDivisionError: division by zero

,Illiquidity-05
Intercept,0.0321***
,(0.0049)
Post 1,0.0000
,(0.0001)


In [14]:
%%notify
result_post1

[                        RandomEffects Estimation Summary                        
 Dep. Variable:         Illiquidity-05   R-squared:                     3.775e-08
 Estimator:              RandomEffects   R-squared (Between):          -7.022e-07
 No. Observations:             6556442   R-squared (Within):            1.917e-08
 Date:                Tue, Jul 07 2020   R-squared (Overall):          -2.559e-07
 Time:                        09:57:56   Log-likelihood                  1.99e+06
 Cov. Estimator:            Unadjusted                                           
                                         F-statistic:                      0.2475
 Entities:                          87   P-value                           0.6189
 Avg Obs:                    7.536e+04   Distribution:               F(1,6556440)
 Min Obs:                    7.244e+04                                           
 Max Obs:                    7.563e+04   F-statistic (robust):             0.1256
                

<IPython.core.display.Javascript object>